In [1]:
import tensorflow as tf
import numpy as np

tf.enable_eager_execution()

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


position encoding + [convolution-layer × # + self-attention-layer + feed-forward-layer]

> We use depthwise separable convolutions (Chollet, 2016) (Kaiser et al., 2017) rather than traditional ones, as we observe that it is memory efficient and has better generalization.

これはkerasが持ってくれている - [SeparableConv2D](https://keras.io/layers/convolutional/#separableconv2d)

> For the self-attention-layer, we adopt the multi-head attention mechanism

これは自前実装が必要そう

勉強だから自分で実装するけど、答えは[ここ](https://github.com/tensorflow/tensor2tensor/blob/master/tensor2tensor/layers/common_attention.py#L2557)

> A positional encoding is added to the input at the beginning of each encoder layer consisting of sin and cos functions at varying wavelengths, as defined in (Vaswani et al., 2017a). 

これも自前かな？元論文にあたる

tensor2tensorに[実装](https://github.com/tensorflow/tensor2tensor/blob/ed9e3bdfd0292d4b6e5b1a1bf272146c8e2f5e9f/tensor2tensor/layers/common_attention.py#L386)がある。これ使わせてもらいたいなぁ

tensor2tensor自体をpip installしてimportできちゃった、超横着

(これやりだすとmulti-head attentionだってimportできちゃうんじゃないのか？勉強したいから自分で書くつもりだけど…)

> Each of these basic operations (conv/self-attention/ffn) is placed inside a residual block

これも自前かな？

ドロップアウト系は後で見直しが必要だな

layernormとかresidual networkとかはtransformerでもやっている

In [4]:
import tensor2tensor.layers.common_attention

In [5]:
tensor2tensor.layers.common_attention.get_timing_signal_1d?

In [82]:
# Scaled Dot-Product Attention

d_k = 20
d_v = 24
n = 5

Q = np.random.randn(n, d_k).astype(np.float32)
K = np.random.randn(n, d_k).astype(np.float32)
V = np.random.randn(n, d_v).astype(np.float32)

def dot_product_attention(Q, K, V):
    d_k = tf.cast(K.shape[1], tf.float32)
    return tf.matmul(tf.nn.softmax(
        tf.matmul(Q, tf.transpose(K)) / tf.square(d_k)), V)

In [83]:
dot_product_attention(Q, K, V)

<tf.Tensor: id=5051, shape=(5, 24), dtype=float32, numpy=
array([[-0.06497877, -0.26551306, -0.4132626 , -0.26437664,  0.20844099,
        -0.58506936, -0.34095284, -0.5295125 , -0.1359812 , -0.21008614,
        -0.7925345 , -0.04303828, -0.2600369 , -0.40712565, -0.31213376,
        -0.33738914,  0.0243742 ,  0.05142229, -0.15877959,  0.50202566,
        -0.17281958, -0.3236704 ,  0.45624834, -0.448968  ],
       [-0.06590389, -0.26416665, -0.4176786 , -0.26434335,  0.20476946,
        -0.5923201 , -0.3418126 , -0.5398475 , -0.13728186, -0.21365632,
        -0.78682137, -0.02977782, -0.26360416, -0.4075202 , -0.31178504,
        -0.3322509 ,  0.02127411,  0.04507902, -0.13703358,  0.51651335,
        -0.17079738, -0.32865644,  0.44824174, -0.45635408],
       [-0.06585614, -0.2681728 , -0.41098645, -0.2697986 ,  0.2002095 ,
        -0.58859086, -0.33531162, -0.5287464 , -0.13438258, -0.21499233,
        -0.7892546 , -0.04245362, -0.2618675 , -0.400478  , -0.31430125,
        -0.332247

In [87]:
# d_modelってなんぞ？embeddingの出力、attentnionの入力と出力

class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, num_heads, d_k, d_v, **kwargs):
        self._num_heads = num_heads
        self._d_k = d_k
        self._d_v = d_v
        super(MultiHeadAttention, self).__init__(**kwargs)

    def build(self, input_shape):
        Q_shape, K_shape, V_shape = input_shape
        d_model = Q_shape[-1]
        
        W_Q, W_K, W_V = [], [], []
        
        # 次元がよくわからん
        # tensor2tensorより
        # query_ antecedent: a Tensor with shape [batch, length_q, channels]
        # channelって…
        # dotproductのところでQはd_kって書いてあっったけど、
        # d_kだとW_Q(d_model, d_k)と内積とれない…？
        # あと、d_modelは入力と書いてあったけど、じゃぁd_kはどっから来たんだ？
        
        # Instead of performing a single attention function with
        # dmodel-dimensional keys, values and queries,
        # we found it beneficial to linearly project the queries,
        # keys and values h times with different, learned linear
        # projections to dk, dk and dv dimensions, respectively. 
        
        # ここらへんか？dot-productをd_model = d_k = d_vの設定で
        # 一回だけ計算するよりも、ってこと？
        # 「d_k、d_k、d_vへの異なる学習可能な線形写像」
        
        # length_qとかがわからん
        # このattentionも感覚的に分かるようなわからないような…
        
        for idx in range(self._num_heads):
            W_Q.append(tf.get_variable(
                'W_Q_{}'.format(idx), [d_model, self._d_k]))
            W_K.append(tf.get_variable(
                'W_K_{}'.format(idx), [d_model, self._d_k]))
            W_V.append(tf.get_variable(
                'W_V_{}'.format(idx), [d_model, self._d_v]))

        self._W_Q, self._W_K, self._W_V = W_Q, W_K, W_V
        self._W_O = tf.get_variable(
            'W_O', [self._num_heads * self._d_v, d_model])
        
        return super(MultiHeadAttention, self).build(input_shape)
    
    def call(self, x):
        Q, K, V = x
        heads = []
        
        for idx in range(self._num_heads):
            a = tf.matmul(Q, self._W_Q[idx])
            attention = dot_product_attention(
                a,
                tf.matmul(K, self._W_K[idx]),
                tf.matmul(V, self._W_V[idx]))
            heads.append(attention)
        
        # (batch_size, h * d_v)
        heads = tf.concat(heads, 1)
        
        # (batch_size, d_model)
        return tf.matmul(heads, self._W_O)
    
    def compute_output_shape(self, input_shape):
        return input_shape[0]

In [88]:
d_k = 20
d_v = 24
d_model = 12
n = 5

inputs = [tf.keras.layers.Input(shape=(d_model,)) for _ in range(3)]
multi_head_attention = MultiHeadAttention(8, d_k, d_v)(inputs)

model = tf.keras.models.Model(
    inputs=inputs, outputs=multi_head_attention)
model.compile(
    optimizer=tf.train.GradientDescentOptimizer(0.001),
    loss='mse')

In [89]:
Q, K, V = [np.random.randn(n, d_model).astype(np.float32) for _ in range(3)]

a = model.predict([Q, K, V])

In [90]:
a.shape

(5, 12)